In [1]:
!pip install dmba
import dmba
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from dmba import regressionSummary

no display found. Using non-interactive Agg backend


In [2]:
df = pd.read_csv('ToyotaCorolla.csv')

In [3]:
df.head()

,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Parking_Assistant,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,1,1,0,0,0,1,0,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0
2,3,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,9,2002,41711,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,7,2002,48000,Diesel,90,0,...,0,1,0,0,0,1,0,0,0,0
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,3,2002,38500,Diesel,90,0,...,1,1,0,1,0,1,0,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1436 entries, 0 to 1435
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 1436 non-null   int64 
 1   Model              1436 non-null   object
 2   Price              1436 non-null   int64 
 3   Age_08_04          1436 non-null   int64 
 4   Mfg_Month          1436 non-null   int64 
 5   Mfg_Year           1436 non-null   int64 
 6   KM                 1436 non-null   int64 
 7   Fuel_Type          1436 non-null   object
 8   HP                 1436 non-null   int64 
 9   Met_Color          1436 non-null   int64 
 10  Color              1436 non-null   object
 11  Automatic          1436 non-null   int64 
 12  CC                 1436 non-null   int64 
 13  Doors              1436 non-null   int64 
 14  Cylinders          1436 non-null   int64 
 15  Gears              1436 non-null   int64 
 16  Quarterly_Tax      1436 non-null   int64 


In [5]:
predictors = ['Age_08_04','KM','Fuel_Type','HP','Automatic','Doors','Quarterly_Tax','Mfr_Guarantee','Guarantee_Period','Airco','Automatic_airco','CD_Player','Powered_Windows','Sport_Model','Tow_Bar']
outcome = 'Price'

In [6]:
# partition data
X = pd.get_dummies(df[predictors], drop_first=True)
y = df[outcome]
minY = y.min()
rangeY = (y.max() - y.min())
y = (y - minY) / rangeY
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.4, random_state = 1)

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
scaler = MinMaxScaler()

In [9]:
# standardize the data
scaler.fit(X_train[['Age_08_04','KM','HP','Automatic','Doors','Quarterly_Tax','Mfr_Guarantee','Guarantee_Period','Airco','Automatic_airco','CD_Player','Powered_Windows','Sport_Model','Tow_Bar']])

MinMaxScaler()

In [10]:
priceNorm = pd.concat([pd.DataFrame(scaler.transform(df[['Age_08_04','KM','HP','Automatic','Doors','Quarterly_Tax','Mfr_Guarantee','Guarantee_Period','Airco','Automatic_airco','CD_Player','Powered_Windows','Sport_Model','Tow_Bar']]),
                                  columns = ['zAge_08_04','zKM','zHP','zAutomatic','zDoors','zQuarterly_Tax','zMfr_Guarantee','zGuarantee_Period','zAirco','zAutomatic_airco','zCD_Player','zPowered_Windows','zSport_Model','zTow_Bar']),
                     df[['Price']]], axis = 1)

In [11]:
X_trainNorm = priceNorm.iloc[X_train.index]
X_validNorm = priceNorm.iloc[X_valid.index]

In [12]:
X_train = X_trainNorm[['zAge_08_04','zKM','zHP','zAutomatic','zDoors','zQuarterly_Tax','zMfr_Guarantee','zGuarantee_Period','zAirco','zAutomatic_airco','zCD_Player','zPowered_Windows','zSport_Model','zTow_Bar']]
X_valid = X_validNorm[['zAge_08_04','zKM','zHP','zAutomatic','zDoors','zQuarterly_Tax','zMfr_Guarantee','zGuarantee_Period','zAirco','zAutomatic_airco','zCD_Player','zPowered_Windows','zSport_Model','zTow_Bar']]

In [13]:
# fit the model 
model_1 = MLPRegressor(hidden_layer_sizes=(2), activation='logistic', solver='lbfgs', random_state=1)

In [14]:
model_1.fit(X_train, y_train)

MLPRegressor(activation='logistic', hidden_layer_sizes=2, random_state=1,
             solver='lbfgs')

In [15]:
y_pred_train = model_1.predict(X_train)

In [16]:
y_pred_train = y_pred_train * rangeY + minY
y_train = y_train * rangeY + minY

In [17]:
# evaluate the model
regressionSummary(y_train, y_pred_train)


Regression statistics

                      Mean Error (ME) : -0.5884
       Root Mean Squared Error (RMSE) : 1108.9802
            Mean Absolute Error (MAE) : 827.7687
          Mean Percentage Error (MPE) : -1.1549
Mean Absolute Percentage Error (MAPE) : 8.0999


In [18]:
y_pred = model_1.predict(X_valid)

In [19]:
y_pred = y_pred * rangeY + minY
y_valid = y_valid * rangeY + minY

In [20]:
# evaluate the model
regressionSummary(y_valid, y_pred)


Regression statistics

                      Mean Error (ME) : 67.3723
       Root Mean Squared Error (RMSE) : 1104.3089
            Mean Absolute Error (MAE) : 872.9668
          Mean Percentage Error (MPE) : -0.5376
Mean Absolute Percentage Error (MAPE) : 8.7825


In [21]:
# Question 1: RMSE for training data: 1108.9802, RMSE for validation data: 1104.3089

In [22]:
# partition data
X = pd.get_dummies(df[predictors], drop_first=True)
y = df[outcome]
minY = y.min()
rangeY = (y.max() - y.min())
y = (y - minY) / rangeY
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.4, random_state = 1)

In [23]:
# standardize the data
scaler.fit(X_train[['Age_08_04','KM','HP','Automatic','Doors','Quarterly_Tax','Mfr_Guarantee','Guarantee_Period','Airco','Automatic_airco','CD_Player','Powered_Windows','Sport_Model','Tow_Bar']])

MinMaxScaler()

In [24]:
priceNorm = pd.concat([pd.DataFrame(scaler.transform(df[['Age_08_04','KM','HP','Automatic','Doors','Quarterly_Tax','Mfr_Guarantee','Guarantee_Period','Airco','Automatic_airco','CD_Player','Powered_Windows','Sport_Model','Tow_Bar']]),
                                  columns = ['zAge_08_04','zKM','zHP','zAutomatic','zDoors','zQuarterly_Tax','zMfr_Guarantee','zGuarantee_Period','zAirco','zAutomatic_airco','zCD_Player','zPowered_Windows','zSport_Model','zTow_Bar']),
                     df[['Price']]], axis = 1)

In [25]:
X_trainNorm = priceNorm.iloc[X_train.index]
X_validNorm = priceNorm.iloc[X_valid.index]

In [26]:
X_train = X_trainNorm[['zAge_08_04','zKM','zHP','zAutomatic','zDoors','zQuarterly_Tax','zMfr_Guarantee','zGuarantee_Period','zAirco','zAutomatic_airco','zCD_Player','zPowered_Windows','zSport_Model','zTow_Bar']]
X_valid = X_validNorm[['zAge_08_04','zKM','zHP','zAutomatic','zDoors','zQuarterly_Tax','zMfr_Guarantee','zGuarantee_Period','zAirco','zAutomatic_airco','zCD_Player','zPowered_Windows','zSport_Model','zTow_Bar']]

In [27]:
# fit the model 
model_2 = MLPRegressor(hidden_layer_sizes=(5,5), activation='logistic', solver='lbfgs', random_state=1)

In [28]:
model_2.fit(X_train, y_train)

MLPRegressor(activation='logistic', hidden_layer_sizes=(5, 5), random_state=1,
             solver='lbfgs')

In [29]:
y_pred_train = model_2.predict(X_train)

In [30]:
y_pred_train = y_pred_train * rangeY + minY
y_train = y_train * rangeY + minY

In [31]:
# evaluate the model
regressionSummary(y_train, y_pred_train)


Regression statistics

                      Mean Error (ME) : -0.9852
       Root Mean Squared Error (RMSE) : 1098.7139
            Mean Absolute Error (MAE) : 817.6844
          Mean Percentage Error (MPE) : -1.1179
Mean Absolute Percentage Error (MAPE) : 8.0340


In [32]:
y_pred = model_2.predict(X_valid)

In [33]:
y_pred = y_pred * rangeY + minY
y_valid = y_valid * rangeY + minY

In [34]:
# evaluate the model
regressionSummary(y_valid, y_pred)


Regression statistics

                      Mean Error (ME) : 64.5413
       Root Mean Squared Error (RMSE) : 1084.7084
            Mean Absolute Error (MAE) : 856.1120
          Mean Percentage Error (MPE) : -0.5898
Mean Absolute Percentage Error (MAPE) : 8.6758


In [35]:
# Question 2: Increasing the number of both nodes and layers decreased the RMSE for both the training and validation data. 
# For this application, it seems that having 2 layers of 5 nodes each is preferable to a single layer with two nodes.